In [13]:
from _utils import wltest_coloring, graph_generator, derive_adjacency,check_symmetric_decomposable
import matplotlib.pyplot as plt 
import networkx as nx 
import torch
import numpy as np
import os
import json


## Load Problem Information

In [14]:
o_path = 'MILP/Bin Packing Problem/Bin Packing Problem_[Warehouse Storage Optimization]/'
name = 'model.lp'
path = o_path + name


The content of lp file ：
```
Minimize
  y[0] + y[1] + y[2]
Subject To
 item_0_placement: x[0,0] + x[0,1] + x[0,2] = 1
 item_1_placement: x[1,0] + x[1,1] + x[1,2] = 1
 item_2_placement: x[2,0] + x[2,1] + x[2,2] = 1
 item_3_placement: x[3,0] + x[3,1] + x[3,2] = 1
 item_4_placement: x[4,0] + x[4,1] + x[4,2] = 1
 bin_0_capacity: 8.6600389009 x[0,0] + 9.0341843686 x[1,0]
   + 1.7299075019 x[2,0] + 0.1213382457 x[3,0] + 3.9651754001 x[4,0]
   - 4.8654483379 y[0] <= 0
 bin_1_capacity: 8.6600389009 x[0,1] + 9.0341843686 x[1,1]
   + 1.7299075019 x[2,1] + 0.1213382457 x[3,1] + 3.9651754001 x[4,1]
   - 4.8654483379 y[1] <= 0
 bin_2_capacity: 8.6600389009 x[0,2] + 9.0341843686 x[1,2]
   + 1.7299075019 x[2,2] + 0.1213382457 x[3,2] + 3.9651754001 x[4,2]
   - 4.8654483379 y[2] <= 0
Bounds
Binaries
 x[0,0] x[0,1] x[0,2] x[1,0] x[1,1] x[1,2] x[2,0] x[2,1] x[2,2] x[3,0]
 x[3,1] x[3,2] x[4,0] x[4,1] x[4,2] y[0] y[1] y[2]
End

```

There are in total 18 variables and 8 constraints. To avoid ambiguity of bipartite graph representation, we represent the boundary of variables as some constraint nodes. 

Therefore, there should be in total 18+8+2*18 = 62 constraints.

In [15]:
info1 = graph_generator(path)
A1 = info1[0] 
f1 = info1[1]#variable
c1 = info1[2]#constraint
Adj = derive_adjacency(A1)
Adj.shape

torch.Size([80, 80])

Now, given another problem with the following lp file

name: answer_concise_eoe_claude-3-opus-20240229.lp

file content:

```
\ Model Bin Packing
\ LP format - for model browsing. Use MPS format to capture full model detail.
\ Model Bin Packing
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  y_0 + y_1 + y_2
Subject To
 item_0_assignment: x_0_0 + x_0_1 + x_0_2 = 1
 item_1_assignment: x_1_0 + x_1_1 + x_1_2 = 1
 item_2_assignment: x_2_0 + x_2_1 + x_2_2 = 1
 item_3_assignment: x_3_0 + x_3_1 + x_3_2 = 1
 item_4_assignment: x_4_0 + x_4_1 + x_4_2 = 1
 bin_0_capacity: 8.6600389009 x_0_0 + 9.0341843686 x_1_0
   + 1.7299075019 x_2_0 + 0.1213382457 x_3_0 + 3.9651754001 x_4_0
   - 4.8654483379 y_0 <= 0
 bin_1_capacity: 8.6600389009 x_0_1 + 9.0341843686 x_1_1
   + 1.7299075019 x_2_1 + 0.1213382457 x_3_1 + 3.9651754001 x_4_1
   - 4.8654483379 y_1 <= 0
 bin_2_capacity: 8.6600389009 x_0_2 + 9.0341843686 x_1_2
   + 1.7299075019 x_2_2 + 0.1213382457 x_3_2 + 3.9651754001 x_4_2
   - 4.8654483379 y_2 <= 0
Bounds
Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2 x_3_0 x_3_1 x_3_2
 x_4_0 x_4_1 x_4_2 y_0 y_1 y_2
End

```


Notice that the models in two .lp files should be the same

In [16]:
name2 = 'answer_concise_eoe_claude-3-opus-20240229.lp'
path2 = o_path+name2

info2 = graph_generator(path2)
A2 = info2[0] 
f2 = info2[1]#variable
c2 = info2[2]#constraint
Adj2 = derive_adjacency(A2)



In [17]:
from _utils import wltest_coloring_two,check_symmetric_decomposable
from _equivalence_detection_utils import check_color_equivalence,check_answer_name,get_valid_path_list

color1,color_cons1,color_var1,color2,color_cons2,color_var2  = wltest_coloring_two(c1,f1,A1,c2,f2,A2)

In [18]:

problem_name = 'Bin Packing Problem'
problem_type = 'MILP'
problem_dom = 'Bin Packing Problem_[Warehouse Storage Optimization]'
answer_type = check_answer_name(problem_name,problem_dom,problem_type)
if check_color_equivalence(color_cons1, color_cons2) and check_color_equivalence(color_var1, color_var2):
    if answer_type == 'WL-determinable':
        print('WL-determinable Equivalence')
    elif answer_type == 'Symmetric Decomposable':
        Adj2= derive_adjacency(A2)
        test_state = check_symmetric_decomposable(color2,Adj2)[0]
        print('check symmetric decomposability for tested graph:',)
        print('check color equivalence:', test_state)
        if test_state:
            print('Symmetric Decomposable Equivalence')
        else:
            print('Not determinable due to test graph indecomposable')
elif not check_color_equivalence(color1, color2):
    print('Not the same color')

check symmetric decomposability for tested graph:
check color equivalence: True
Symmetric Decomposable Equivalence
